In [38]:
import pandas as pd

df = pd.read_csv('csv_outputs/latest_tvl_app_trends_by_token.csv')

In [39]:
name = 'Radiant'
chain = 'Arbitrum'
trailing_num_days = 30

In [40]:
ndf = df[
        (df['name'].str.lower() == name.lower())
        & (df['chain'].str.lower() == chain.lower())
        # & (df['chain'].str.lower() == chain.lower())
]
# create a new column with ranks of "dt"
ndf['date_rank'] = ndf['date'].rank(method='min', ascending=False)

ndf = ndf[ndf['date_rank'] <= trailing_num_days]
# display(ndf)
# print(ndf.columns)

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_81505/3393162191.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndf['date_rank'] = ndf['date'].rank(method='min', ascending=False)


In [42]:
piv_df = ndf.groupby(['chain','protocol','token']).agg({'net_dollar_flow_latest_price':'sum'})
piv_df['share_of_total'] = piv_df['net_dollar_flow_latest_price'] / piv_df.groupby(['chain','protocol'])['net_dollar_flow_latest_price'].transform('sum')


display(piv_df)

net_dollar_flow_latest_price  share_of_total
chain    protocol token                                              
Arbitrum radiant  DAI                    5.141380e+06        0.169288
                  USDC                   1.112935e+07        0.366451
                  USDT                   2.485402e+06        0.081836
                  WBTC                   2.121401e+06        0.069850
                  WETH                   9.493130e+06        0.312576